In [ ]:
# pip install


In [1]:
# set working directory
import os
os.chdir(r'C:\Users\a1bg532573\repo\Bulgarian-AI-Folktales\preprocesing')
# imports 
from langchain.document_loaders.pdf import PyPDFDirectoryLoader  # Importing PDF loader from Langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter  # Importing text splitter from Langchain

from langchain_openai import OpenAIEmbeddings # Importing OpenAI embeddings from Langchain
from langchain.schema import Document  # Importing Document schema from Langchain
from langchain_chroma import Chroma  # Importing Chroma vector store from Langchain
from dotenv import load_dotenv # Importing dotenv to get API key from .env file
from langchain.chat_models import ChatOpenAI

import os  # Importing os module for operating system functionalities
import shutil  # Importing shutil module for high-level file operations

import json
from langchain.schema import Document

from collections import defaultdict

import time
from typing import List, Tuple
from langchain.schema import Document
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
import os
import deepl

### 1. Load documents and clean duplicates


In [2]:
def load_documents(DATA_PATH):
    """
    Load text documents from JSON files in the specified directory.

    Returns:
        List of Document objects: Loaded text documents represented as Langchain Document objects.
    """
    documents = []

    # Iterate over each file in the directory
    for filename in os.listdir(DATA_PATH):
        if filename.endswith(".json"):
            file_path = os.path.join(DATA_PATH, filename)
            
            # Open and read each JSON file
            with open(file_path, 'r', encoding='utf-8') as json_file:
                data = json.load(json_file)
                
                # Extract book, author, and stories from JSON
                book_name = data.get("book", "")
                author = data.get("author", "")
                stories = data.get("stories", [])

                # Iterate through each story and create Document objects
                for story in stories:
                    if "story" in story or "author" in story:
                        story_title = story.get("story") or story.get("author")
                        story_content = story.get("content", "")

                        # Append the story title to the beginning of the story content
                        combined_content = f"{story_title}: {story_content}"

                        # Create a Document object with metadata
                        document = Document(
                            page_content=combined_content,
                            metadata={
                                "book": book_name,
                                "author": author,
                                "story": story_title
                            }
                        )
                        # Append the document to the list of documents
                        documents.append(document)

    return documents

def keep_unique_documents(documents):
    """
    Keep only one unique record for each duplicate, removing redundant records.

    Args:
        documents (list): List of Document objects.

    Returns:
        List of Document objects: List with unique documents, retaining one instance for each duplicate.
    """
    content_tracker = {}
    unique_documents = []

    # Track each document's combined content and keep only the first occurrence
    for doc in documents:
        content = doc.page_content.strip().lower()  # Normalize for comparison
        if content not in content_tracker:
            # Add the first occurrence to the unique list
            content_tracker[content] = doc
            unique_documents.append(doc)

    return unique_documents

DATA_PATH = 'output_json_files'
documents = load_documents(DATA_PATH)  # Load documents from a source
unique_documents = keep_unique_documents(documents)  # Keep only unique documents

### 2. Building a small quick token size function


In [3]:
import tiktoken

def count_tokens(text: str, model: str = "gpt-3.5-turbo") -> int:
    """
    Count the number of tokens in a given text using the specified model's tokenizer.

    Args:
        text (str): The input text to tokenize.
        model (str): The name of the model to use for tokenization (default: "gpt-3.5-turbo").

    Returns:
        int: The number of tokens in the text.
    """
    model = "gpt-4o-mini"
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

### 3. Sort documents by token size and print first and last examples

In [4]:
# sort unique_documents by unique_documents[i].page_content token size
sorted_documents = sorted(unique_documents, key=lambda x: count_tokens(x.page_content), reverse=True)

# Print the first 10 documents
for doc in sorted_documents[:10]:
    print(f"Document: {doc.metadata['story'][:30]}... | Tokens: {count_tokens(doc.page_content)}")
print("---------------")
# Print last 10 documents
for doc in sorted_documents[-30:]:
    print(f"Document: {doc.page_content}... | Tokens: {count_tokens(doc.page_content)}")

Document: повест за една гора... | Tokens: 75356
Document: срещата на най-големия с ламят... | Tokens: 18720
Document: щетинското ханче... | Tokens: 10889
Document: в тронната зала... | Tokens: 8410
Document: веселият монах... | Tokens: 7511
Document: босата команда... | Tokens: 6840
Document: гарван грачи... | Tokens: 5535
Document: юнакът със звезда на челото и ... | Tokens: 5135
Document: юнакът със звезда на челото и ... | Tokens: 5129
Document: врабчетата на стрина дойна... | Tokens: 4829
---------------
Document: жените или мъжете са повече на земята?: където ходели настрадин ходжа и хитър петър, все се нещо препирали, изпитвали, надлъгвали. като вървели веднъж. хитър петър разправял, че жените са повече на земното кълбо, а ходжата казвал, че мъжете са повече. слушал го, слушал хитър петър, па му рекъл — не си прав, ходжа! жените са повече, защото има мъже, които слушат жените си затова и те се числят към жените!... | Tokens: 141
Document: изгубил доверие: в селото на хитър петър

### 4. Sort docments in bins based on token counts
#### 1. Dropping binned_documents['0-60'] as it is too small and possible left noise from parsing
#### 2. Taking bin ['60-1000'] as full ebedding
#### 3. Taking bin ['1000-'float('inf')] as chunk embeding

In [5]:
def sort_documents_into_bins(documents, bin_ranges):
    """
    Sort documents into bins based on their token counts.
    
    Args:
        documents (list): List of Document objects.
        bin_ranges (list): List of tuples representing the start and end of each bin range.
    
    Returns:
        dict: A dictionary with bin ranges as keys and sorted lists of documents as values.
    """
    bins = {f"{start}-{end}": [] for start, end in bin_ranges}
    
    for doc in documents:
        token_count = count_tokens(doc.page_content)
        for start, end in bin_ranges:
            if start <= token_count < end:
                bins[f"{start}-{end}"].append((doc, token_count))
                break
    
    # Sort documents within each bin by token count (descending order)
    for bin_range in bins:
        bins[bin_range].sort(key=lambda x: x[1], reverse=True)
    
    return bins

# Define bin ranges
bin_ranges = [(0, 60), (60, 1000), (1000, float('inf'))]

# Sort documents into bins
binned_documents = sort_documents_into_bins(unique_documents, bin_ranges)

# Print the number of documents in each bin
for bin_range, docs in binned_documents.items():
    print(f"Bin {bin_range}: {len(docs)} documents")

# Print example documents from each bin
for bin_range, docs in binned_documents.items():
    print(f"\nBin {bin_range}:")
    for doc, token_count in docs[:20]:  # Print first 20 documents in each bin
        print(f"  Document: {doc.metadata['story'][:30]}... | Tokens: {token_count}")
    if len(docs) > 20:
        print("  ...")
    # Print last 20 documents in each bin
    print(f"  Last 20 documents in {bin_range}:")
    for doc, token_count in docs[-20:]:
        print(f"    Document: {doc.metadata['story'][:30]}... | Tokens: {token_count}")

Bin 0-60: 16 documents
Bin 60-1000: 398 documents
Bin 1000-inf: 433 documents

Bin 0-60:
  Document: елин пелин... | Tokens: 33
  Document: леда милева... | Tokens: 31
  Document: елин пелин... | Tokens: 27
  Document: сава попов... | Tokens: 25
  Document: горска хубавица... | Tokens: 23
  Document: ангел каралийчев... | Tokens: 18
  Document: ангел каралийчев... | Tokens: 17
  Document: ангел каралийчев... | Tokens: 17
  Document: ангел каралийчев... | Tokens: 17
  Document: ангел каралийчев... | Tokens: 16
  Document: ангел каралийчев... | Tokens: 13
  Document: ангел каралийчев... | Tokens: 13
  Document: през планини и морета... | Tokens: 13
  Document: елин пелин... | Tokens: 12
  Document: елин пелин... | Tokens: 12
  Document: елин пелин... | Tokens: 11
  Last 20 documents in 0-60:
    Document: елин пелин... | Tokens: 33
    Document: леда милева... | Tokens: 31
    Document: елин пелин... | Tokens: 27
    Document: сава попов... | Tokens: 25
    Document: горска хубавица... |

### 5. Custom Contextual Enhancing embedding for ['60-1000'] bin (**Small Stories**) and ['1000-inf'] bin (**Large Stories**) with GPT-4o-mini

- Timeout mechanism to avoid hitting TPM limit of gpt-4o-mini (200 000 TPM) - keep track of the number of tokens sent to the LLM and pause for 1 minute the process if the limit is about to be reached
- Handling large documents:
1. Document size: Handles texts ranging from 1,000 to 20,000 tokens.
2. Dynamic chunking: Uses a text splitter to divide documents into ~1,000 token chunks with 200 token overlap. Chunk count adapts to document length.
3. Contextual embedding: Applies `get_contextual_embedding()` with `prompt_type="contextual"` to each chunk, providing context within the full document.
4. Document-level enhancement: Calls `get_contextual_embedding()` with `prompt_type="enhancing"` once for the entire document.
5. Information combination: Appends both chunk-specific contextual info and document-level enhancing info to each chunk, resulting in multiple chunks with shared enhancing info but unique contextual details.

In [22]:
# Constants
TEST_CHROMA_PATH = "vector_store_test"
TPM_LIMIT = 200000
PAUSE_TIME = 80  # seconds

with open('../config.json') as config_file:
    config = json.load(config_file)
    os.environ['OPENAI_API_KEY'] = config['OPENAI_API_KEY']

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Initialize components
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
# Set up ChatOpenAI model
llm = ChatOpenAI(
    model="gpt-4o-mini",  # Use a smaller model for faster response times
    # model="gpt-4o",  # Use a smaller model for faster response times
    api_key=OPENAI_API_KEY,
    temperature=0,       # Increase temperature for more creativity
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

def translate_in_bulgarian(text, llm=None):

    if llm is not None:
        transation_prompt =f"""Translate the following text into Bulgarian. Please adhere to these guidelines:

        1. Maintain the original meaning and tone.
        2. Avoid word-for-word translation; focus on natural Bulgarian phrasing.
        3. Use diverse vocabulary to prevent repetition of words or phrases.
        4. Preserve any formatting or structure present in the original text.
        5. If there are specialized terms, provide the most appropriate Bulgarian equivalent.
        6. Ensure the translation aligns with Bulgarian cultural context where applicable.

        <text>
        {text}
        </text>

        Please provide only the Bulgarian translation without any additional comments."""

        response = llm.invoke(transation_prompt)
        response = response.content
    else:
        # use DeepL 
        deepl_auth_key = "3afd0c9a-0aa4-9220-6227-778e77bcfc64:fx"  # Replace with your key
        translator = deepl.Translator(deepl_auth_key)

        result = translator.translate_text(text, target_lang="BG")
        response = result.text

    return response


def get_contextual_embedding(doc: Document, llm: ChatOpenAI, prompt_type: str, full_text: str = None) -> str:
    """
    Generate a contextual embedding for a document using the LLM.
    
    Args:
        doc (Document): The document or chunk to embed.
        llm (ChatOpenAI): The language model to use for generating context.
        prompt_type (str): Type of prompt to use ("contextual" or "enhancing").
        full_text (str): The full text of the document (used for contextual embedding).
    
    Returns:
        str: The contextual embedding as a string.
    """
    if prompt_type == "contextual":
        prompt = f"""
        Here is the chunk we want to situate within the whole document
        <chunk>
        {doc.page_content}
        </chunk>

        Here is the content of the whole document
        <document>
        {full_text}
        </document>

        Please give a short succinct context to situate this chunk within the overall document for the purposes of improving search retrieval of the chunk.
        Answer only with the succinct context and nothing else. Answer in Bulgarian.
        """
    elif prompt_type == "enhancing":
        prompt = f"""
        Based on the following story, provide a concise summary including:
        1. A brief synopsis of the story
        2. The main characters
        3. The setting or environment
        4. The moral or main message of the story
        <story>
        Story: {doc.page_content}
        </story>

        Respond in a concise paragraph format: "**brief summary of the story:** ..., **main characters:** ..., **setting:** ..., **moral:** ..." *... are placeholders for the actual values you will provide after reading the story. Please give a succinct answer to augment the document for the purposes of improving search retrieval of the story.
        Separate every value in a separate line with a new line character.
        """
    
    response = llm.invoke(prompt)
    return response.content

def extract_main_characters(enhancing_info):
    lines = enhancing_info.split('\n')
    for line in lines:
        if line.startswith("**главни герои:**"):
            return line.split(":**", 1)[1].strip()
    return ""

def embed_documents(documents: List[Tuple[Document, int]], llm: ChatOpenAI, embeddings, chroma_path: str) -> None:
    total_tokens = 0
    start_time = time.time()
    combined_docs = []
    doc_id = 1  # Start with document ID 1

    for idx, (doc, token_count) in enumerate(documents, 1):
        print(f"Processing document {idx}/{len(documents)}: {doc.metadata['story'][:30]}...")
        
        if total_tokens + token_count > TPM_LIMIT:
            elapsed_time = time.time() - start_time
            if elapsed_time < PAUSE_TIME:
                pause_duration = PAUSE_TIME - elapsed_time
                print(f"Approaching TPM limit. Pausing for {pause_duration:.2f} seconds.")
                time.sleep(pause_duration)
            total_tokens = 0
            start_time = time.time()

        is_large_doc = token_count >= 1000

        if is_large_doc:
            character_count = len(doc.page_content)
            num_chunks = max(2, token_count // 1000)
            chunk_size = (character_count // num_chunks) + 200
            
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=chunk_size,
                chunk_overlap=200,
                length_function=len,
            )

            chunks = text_splitter.split_documents([doc])

            enhancing_info = get_contextual_embedding(doc, llm, prompt_type="enhancing")
            enhancing_info = translate_in_bulgarian(enhancing_info, llm=None) #llm=None use DeepL api
            # enhancing_info = translate_in_bulgarian(enhancing_info, llm)
            main_characters = extract_main_characters(enhancing_info)
            
            for chunk_idx, chunk in enumerate(chunks, 1):
                contextual_info = get_contextual_embedding(chunk, llm, prompt_type="contextual", full_text=doc.page_content)
                combined_content = f"{chunk.page_content}\n\nContextual Information:\n\n{contextual_info}\n\nEnhancing Information:\n\n{enhancing_info}"
                
                contextual_doc = Document(page_content=combined_content, metadata={**doc.metadata, 'main_characters': main_characters})

                combined_docs.append((str(doc_id), contextual_doc))
                doc_id += 1

            print(f"Embedded large document: {doc.metadata['story'][:30]}...")
            print(f"Number of splits: {len(chunks)} for Character count: {character_count} for Tokens {token_count}")
        else:
            enhancing_info = get_contextual_embedding(doc, llm, prompt_type="enhancing")
            enhancing_info = translate_in_bulgarian(enhancing_info, llm=None) # llm=None using DeepL
            # enhancing_info = translate_in_bulgarian(enhancing_info, llm)
            # add main character metadata
            main_characters = extract_main_characters(enhancing_info)
            combined_content = f"{doc.page_content}\n\nEnhancing Information:\n\n{enhancing_info}"
            
            contextual_doc = Document(page_content=combined_content, metadata={**doc.metadata, 'main_characters': main_characters})
            
            combined_docs.append((str(doc_id), contextual_doc))
            doc_id += 1
            print(f"Embedded small document: {doc.metadata['story'][:30]}...")

        total_tokens += token_count
        print(f"Total tokens: {total_tokens}")
    
    db = Chroma.from_documents(
        documents=[doc for _, doc in combined_docs],
        embedding=embeddings,
        persist_directory=chroma_path,
        ids=[id for id, _ in combined_docs]
    )
    print(f"Saved {len(combined_docs)} documents to {chroma_path}.")

# Test with documents from both bins
small_docs = binned_documents['60-1000'][:2]  # Increased to 2 small documents for better testing
large_docs = [binned_documents['1000-inf'][-1]]
test_docs = small_docs + large_docs

embed_documents(test_docs, llm, embeddings, TEST_CHROMA_PATH)
print("Finished embedding all documents and saved to Chroma DB.")

# Verify the IDs
db = Chroma(persist_directory=TEST_CHROMA_PATH, embedding_function=embeddings)
result = db.get(include=['metadatas'])
print("Document IDs:")
print(result['ids'])


Processing document 1/3: трите патенца...
Embedded small document: трите патенца...
Total tokens: 999
Processing document 2/3: сън...
Embedded small document: сън...
Total tokens: 1992
Processing document 3/3: врабчето си иска зърното...
Embedded large document: врабчето си иска зърното...
Number of splits: 2 for Character count: 2523 for Tokens 1009
Total tokens: 3001
Saved 4 documents to vector_store_test.
Finished embedding all documents and saved to Chroma DB.
Document IDs:
['b51a5ca9-5852-47dc-8a76-518726a93bc1', '87a8cc1b-58d6-4146-a101-a59db6e670ed', '0f97b8f7-7fb5-4eb4-a809-5922889cdd8c', '1', '2', '3', '4']


### 6. Testing the retrieval of the stored embeddings

In [23]:
db = Chroma(persist_directory=TEST_CHROMA_PATH)
result = db.get(include=['embeddings'])
result

{'ids': ['b51a5ca9-5852-47dc-8a76-518726a93bc1',
  '87a8cc1b-58d6-4146-a101-a59db6e670ed',
  '0f97b8f7-7fb5-4eb4-a809-5922889cdd8c',
  '1',
  '2',
  '3',
  '4'],
 'embeddings': array([[ 0.06076654,  0.02530093,  0.01129333, ...,  0.0064741 ,
         -0.01222809,  0.03312524],
        [ 0.03261885,  0.0120194 , -0.03974689, ...,  0.00992206,
          0.02024011,  0.04889894],
        [ 0.01968575,  0.01906617, -0.04697545, ...,  0.02292447,
          0.00985696,  0.05542427],
        ...,
        [-0.01562168,  0.02581631, -0.04501637, ...,  0.00038412,
          0.0213298 , -0.04110285],
        [ 0.02068438,  0.04800818, -0.05613336, ...,  0.02189154,
         -0.00986048,  0.04605814],
        [ 0.01803932,  0.05230949, -0.06094486, ...,  0.03790969,
         -0.01234269,  0.05560992]]),
 'metadatas': None,
 'documents': None,
 'uris': None,
 'data': None,
 'included': ['embeddings']}

In [26]:
db.get('4')

{'ids': ['4'],
 'embeddings': None,
 'metadatas': [{'author': 'ангел каралийчев',
   'book': 'Български народни приказки ак',
   'chunk': 2,
   'main_characters': 'Сивото врабче, оградата, огънят, реката, биволът, вълкът, пастирът, мишките и котката.',
   'story': 'врабчето си иска зърното',
   'total_chunks': 2}],
 'documents': ['бивола. — какво приказваш — отговорил вълкът, — додето има такива крехки агънца, що ми трябва жилаво биволско месо! — ще кажа на овчаря да насъска кучетата и те ще ти разкъсат кожуха! — кажи му де! — обърнал се вълкът. — овчарко — викнало отдалече врабчето, — проводи кучетата да изядат вълка! — дордето имам в торбата си мек хляб, защо ми трябва да провождам кучетата да си трошат зъбите с кораво вълче месо — отговорил овчарят. — но аз ще кажа на мишките да ти изгризат торбата! — кажи им де! — мишки — надникнало врабчето в една миша дупка, — излезте да изгризете кожената торба на овчаря! — додето си имаме зърно в житницата, торба няма да гризем — отговорили миш

In [76]:
from IPython.display import display, Markdown, Latex
display(Markdown(db.get('2')["documents"][0]))

сън: разказва кучето джафко знайте ли кое е на света най-сладко? всички ще речете знаем, джафко, знаем. ти месце обичаш. то за теб ще бъде най-сладкото нещо! — лъжете се, братя! няма да отричам, че месце обичам. но не зная нещо, от съня по-сладко. цял ден си лудувал, джафкал си, играл си. хапнал недохапнал, сръбнал недосръбнал — сгушиш се на топло, па заспиш си сладко. и още по-сладки сънища сънуваш. насън всичко може. ходиш, дето искаш. правиш си на воля, каквото желаеш. цял ден ти се щяло нещичко да хапнеш, легнеш вечер гладен и — хоп — насън виждаш богата трапеза. и хапнеш, и сръбнеш. никой ти не виква. никой те не ритва ето оназ вечер какъв сън сънувах вървя с наша данка. стигаме до моста. изведнъж усещам, че някой ме дръпва и така ми казва — пардон, господине! вие нали бяхте стихоплетът джафко? аз се понавъсвам. отвръщам сърдито — не съм стихоплетът, а поетът джафко. за какво ти трябвам? — извинете, моля. едно писмо нося. царят ви го праща. аз питам учуден? — кой цар ми го праща? — царят, що владее кучешкото царство. отварям писмото. чета и не зная от радост къде съм. — драги ми поете — пише ми сам царят, — с наслада ти четох днеска стиховете. и те назначавам придворен учител. ела тук веднага. че на царедворци чакат синовете, за да ги направиш всичките поети! подавам писмото на дана и казвам — на, чети, да видиш каква чест ми правят, та и ти да знаеш как да ме зачиташ. а сега прощавай! при царя отивам. дигам глава гордо. с пратеника тръгвам. пристигаме скоро в школото на царя. там сварвам чедата на видните псета. като ме съзряха, зашепнаха тихо поета, поета! сам царят ме среща. аз му се покланям. — благодаря, царю, за честта голяма. за мене по-драга от таз служба няма. но тясно е тука, о, царю честити, за мойта наука. вънка на открито трябва да излязат моите ученици. и волни да бъдат като горски птици. по двор и градини, в гори и долини да тичат, да скачат. с мене да играят. и аз ще ги уча що трябва да знаят. тука ще се връщат всички изгладнели. та ядене царско за вечер гласете! и царят знак дава! — да бъде, поете! и радост голяма тогава настава. на възбог ме дигат моите ученици и скачат, и викат — учителю драги, жив и здрав бъди ни! по-скоро навънка! по-скоро води ни в царските градини! но един глас страшен най-силно крещеше — навънка! навънка! и аз се събудих. готвачката деша викаше сърдито — вънка, вънка, вънка! легнал ми в леглото! ах, джафко проклети! и тя ме изрита. но аз й прощавам. че тя знае само да пържи кюфтета. затова тъй грубо съня ми прекъсна. но от туй той още по-сладък ми стана. и аз ще го помня, докато живея!

Enhancing Information:

**кратко резюме на историята:** Историята разказва за кучето на име Джафко, което размишлява за сладостта на мечтите в сравнение с любовта си към месото. То мечтае да бъде назначено за кралски учител на децата на краля, наслаждавайки се на свободата и радостта от преподаването на открито. Въпреки това, мечтата му е внезапно прекъсната от готвача, когото в крайна сметка прощава, като още повече цени сладостта на своята мечта.

**главни герои:** Джафко (кучето), кралят, Данка (спътник), Деша (готвачката).

**местоположение:** Историята се развива в приказно кучешко кралство, основно в училището на краля и околните градини.

**поучение:** Историята предава, че мечтите могат да бъдат по-пълноценни и сладки от реалността, а прекъсванията в живота могат да доведат до по-голямо оценяване на тези мечти.

In [77]:
from IPython.display import display, Markdown, Latex
display(Markdown(db.get('3')["documents"][0]))

врабчето си иска зърното: сивото врабче кацнало върху един плет и почнало да си ниже герданче от мънистени зрънца. както нижело, изтървало едно зърно. зърното паднало в тръните, търколило се някъде и се загубило. — хей, плет — изчуруликало врабчето, — дай ми мънистеното зърно или ще кажа на огъня да те изгори! — кажи му де! — отвърнал плетът. — огънчо — хвръкнало врабчето над огъня, — изгори плета! — не ща — отвърнал огънят. — докато си имам сухи букови дървета, много ми е притрябвало да горя трънливия плет и да се бода на тръните му. — ще кажа на реката да те угаси! — кажи й де! — отвърнал огънят. врабчето литнало над реката и зачуруликало. — речице, моля ти се, угаси огъня! — ами — отговорила реката, — много ми е притрябвало да гася огън. додето си имам тия гладки камъчета, които сега броя, що ми трябва да се паря с огън? — ще кажа на бивола да те изпие! — заканило се врабчето. — кажи му де! — биволчо, изпий реката! — кацнало врабчето върху единия рог на бивола. — как не — отвърнал биволът, — аз се напасох с такава росна трева, че ако сръбна и вода — ще ми се надуе коремът и ще се пукне. — тогава ще кажа на вълка да те изяде. — кажи му де! — рекъл биволът. врабчето отишло при вълка в гората. — вълчо — помолило го то, — ела да изядеш бивола. — какво приказваш — отговорил вълкът, — додето има такива крехки агънца, що ми трябва жилаво биволско месо! — ще кажа на овчаря да насъска кучетата и те ще ти разкъсат кожуха! — кажи му де! —

Contextual Information:

Този фрагмент описва как сивото врабче търси изгубеното си зърно и заплашва различни природни елементи (плет, огън, река, бивол, вълк, овчар), за да го получи обратно. В края на историята, след множество заплахи и взаимодействия, врабчето получава зърното и се радва. Този откъс е част от по-голям разказ, който илюстрира темата за взаимопомощ и последователността на действията в природата.

Enhancing Information:

**кратко резюме на историята:** Сивият врабец губи мънисто, докато ги нанизва на ограда, и иска да му бъде върнато, заплашвайки различни елементи на природата. Всяко същество отказва да се подчинява, което води до верижна реакция от заплахи, докато не се намесва котка, предизвиквайки серия от събития, които в крайна сметка водят до това врабецът да си върне мънистото от оградата.  
**главни герои:** Сивият врабец, оградата, огънят, реката, биволът, вълкът, овчарят, мишките и котката.  
**местоположение:** Природна среда с ограда, огън, река и околна дива природа.  
**поучение:** Историята илюстрира безсмислието на заплахите и взаимовръзката в природата, подчертавайки, че сътрудничеството и разбирането са по-ефективни от заплашването.

In [30]:
# from IPython.display import display, Markdown, Latex
# display(Markdown(db.get('82b05836-67ed-49f9-ad25-d51c36c5b038')["documents"][0]))
from IPython.display import display, Markdown, Latex
display(Markdown(db.get('82b05836-67ed-49f9-ad25-d51c36c5b038')["documents"][0]))

бивола. — какво приказваш — отговорил вълкът, — додето има такива крехки агънца, що ми трябва жилаво биволско месо! — ще кажа на овчаря да насъска кучетата и те ще ти разкъсат кожуха! — кажи му де! — обърнал се вълкът. — овчарко — викнало отдалече врабчето, — проводи кучетата да изядат вълка! — дордето имам в торбата си мек хляб, защо ми трябва да провождам кучетата да си трошат зъбите с кораво вълче месо — отговорил овчарят. — но аз ще кажа на мишките да ти изгризат торбата! — кажи им де! — мишки — надникнало врабчето в една миша дупка, — излезте да изгризете кожената торба на овчаря! — додето си имаме зърно в житницата, торба няма да гризем — отговорили мишките. тогава врабчето заплакало. видяла го една котка и го попитала — защо плачеш, врабченце? я млъкни! недей натъжава котешкото ми сърце. — ще млъкна, ако изядеш мишките. изведнъж котката се хвърлила върху мишките, мишките — върху торбата на овчаря, овчарят насъскал кучетата си подир вълка, вълкът подгонил бивола, биволът се навел да изпие реката, реката потекла към огъня, огънят запалил единия край на трънения плет, плетът се уплашил и за да не изгори, намерил мънистеното зърно и го дал на врабчето. врабчето се зарадвало много, кацнало на едно клонче, весело зачуруликало и всички се укротили.

Contextual Information:
Този фрагмент представлява част от историята за врабчето, което търси своето мънистено зърно, и описва взаимодействията между различни животни, включително вълка, бивола, овчаря и мишките, в опитите им да решат конфликта. В края на този епизод, врабчето получава зърното си, след като различните герои реагират на заплахите и молбите му.

Enhancing Information:
**кратко резюме на историята:** Сивото врабче загубва мънистено зърно и заплашва различни същества, за да го получи обратно. След серия от заплахи, всички се обединяват, за да помогнат на врабчето, което в крайна сметка получава зърното си.

**главни герои:** Сивото врабче, плетът, огънят, реката, биволът, вълкът, овчарят, мишките, котката.

**местоположение:** Природна среда с плет, огън, река, гора и полета.

**морал:** Заплахите и конфликти могат да доведат до сътрудничество и решаване на проблеми, а истинската помощ идва от обединението на различни същества.

In [29]:
from IPython.display import display, Markdown, Latex
display(Markdown(db.get('b84b6c61-c932-4d16-9336-9406c62c8e13')["documents"][0]))


бивола. — какво приказваш — отговорил вълкът, — додето има такива крехки агънца, що ми трябва жилаво биволско месо! — ще кажа на овчаря да насъска кучетата и те ще ти разкъсат кожуха! — кажи му де! — обърнал се вълкът. — овчарко — викнало отдалече врабчето, — проводи кучетата да изядат вълка! — дордето имам в торбата си мек хляб, защо ми трябва да провождам кучетата да си трошат зъбите с кораво вълче месо — отговорил овчарят. — но аз ще кажа на мишките да ти изгризат торбата! — кажи им де! — мишки — надникнало врабчето в една миша дупка, — излезте да изгризете кожената торба на овчаря! — додето си имаме зърно в житницата, торба няма да гризем — отговорили мишките. тогава врабчето заплакало. видяла го една котка и го попитала — защо плачеш, врабченце? я млъкни! недей натъжава котешкото ми сърце. — ще млъкна, ако изядеш мишките. изведнъж котката се хвърлила върху мишките, мишките — върху торбата на овчаря, овчарят насъскал кучетата си подир вълка, вълкът подгонил бивола, биволът се навел да изпие реката, реката потекла към огъня, огънят запалил единия край на трънения плет, плетът се уплашил и за да не изгори, намерил мънистеното зърно и го дал на врабчето. врабчето се зарадвало много, кацнало на едно клонче, весело зачуруликало и всички се укротили.

Contextual Information:

Този откъс е част от приказка, в която врабчето се опитва да си върне изгубеното мънистено зърно, като заплашва различни животни и обекти с верига от действия, които водят до хаос, докато накрая всички се успокояват и врабчето получава зърното си обратно.

Enhancing Information:

**brief summary of the story:** Сивото врабче изпуска мънистено зърно в тръните и се опитва да го върне, като заплашва различни елементи от природата и животни, но никой не се съгласява да помогне. Накрая котката се намесва, което предизвиква верижна реакция, водеща до връщането на зърното на врабчето.  
**main characters:** Врабчето, плетът, огънят, реката, биволът, вълкът, овчарят, мишките, котката.  
**setting:** Природна среда с плет, огън, река, гора и животни.  
**moral:** Упоритостта и настойчивостта могат да доведат до успех, дори когато изглежда, че никой не иска да помогне.

In [40]:
response = llm.invoke("""Translate the following text into Bulgarian. Please adhere to these guidelines:

1. Maintain the original meaning and tone.
2. Avoid word-for-word translation; focus on natural Bulgarian phrasing.
3. Use diverse vocabulary to prevent repetition of words or phrases.
4. Preserve any formatting or structure present in the original text.
5. If there are specialized terms, provide the most appropriate Bulgarian equivalent.
6. Ensure the translation aligns with Bulgarian cultural context where applicable.

<text>
Enhancing Information: brief summary of the story: A gray sparrow loses a bead while stringing them on a fence and demands it back, threatening various elements of nature. Each entity refuses to comply, leading to a chain reaction of threats until a cat intervenes, causing a series of events that ultimately results in the sparrow retrieving its lost bead from the fence.

main characters: The gray sparrow, the fence, the fire, the river, the buffalo, the wolf, the shepherd, the mice, and the cat.

setting: The story takes place in a natural environment featuring a fence, a fire, a river, and a forest.

moral: The story illustrates the futility of threats and the interconnectedness of nature, emphasizing that cooperation and understanding are more effective than intimidation.
</text>

Please provide only the Bulgarian translation without any additional comments.""")

display(Markdown(response.content))


Подобряване на информацията: кратко резюме на историята: Сивият врабец губи мънисто, докато ги нанизва на ограда, и иска да му бъде върнато, заплашвайки различни елементи на природата. Всяко същество отказва да се подчинява, което води до верижна реакция от заплахи, докато не се намесва котка, предизвиквайки серия от събития, които в крайна сметка водят до възстановяването на изгубеното мънисто от оградата.

главни герои: Сивият врабец, оградата, огънят, реката, биволът, вълкът, овчарят, мишките и котката.

обстановка: Историята се развива в естествена среда, включваща ограда, огън, река и гора.

морал: Историята илюстрира безсмислието на заплахите и взаимовръзката в природата, подчертавайки, че сътрудничеството и разбирането са по-ефективни от заплашването.

'Морал: Историята илюстрира безсмислието на заплахите и взаимовръзката в природата, подчертавайки, че сътрудничеството и разбирането са по-ефективни от заплахите.'

морал: Историята илюстрира безсмислието на заплахите и взаимосвързаността на природата, подчертавайки, че сътрудничеството и разбирането са по-ефективни от сплашването.'

DeepL translate

In [ ]:
url = "https://api-free.deepl.com/v2/translate"
headers = {
    "Authorization": "DeepL-Auth-Key 3afd0c9a-0aa4-9220-6227-778e77bcfc64:fx",
    "User-Agent": "YourApp/1.2.3"
}

def deepl_translate_batch(text_list):
    data = {
        "text": text_list,
        "target_lang": "EN",
        "source_lang": "BG"
    }
    response = requests.post(url, headers=headers, json=data)
    response_json = json.loads(response.text)
    return [trans['text'] for trans in response_json['translations']]

In [69]:
import deepl

auth_key = "3afd0c9a-0aa4-9220-6227-778e77bcfc64:fx"  # Replace with your key
translator = deepl.Translator(auth_key)

result = translator.translate_text("Hello, world!", target_lang="BG")
print(result.text)  # "Bonjour, le monde !"

Здравей, свят!
